# Multi-Modal Data Analysis Workflow

**ASIST Study 3 Dataset**

#### **Team 000286 Pre-Result Analysis**

Engineer exibited poor skill during the hands on training. It took him a very long time to complete the individual training which resulted in the mission timer running out during the team training section. Once the timer expired, the hands-on training trial was restarted and participants again completed the individual training. Engineer's second attempt still took over 5 minutes to complete individual training. The competency test was then started at 13m remaining on the timer, the engineer took a full 5 minutes to complete the competency test. Experiment team decided that engineer should be allowed to proceed to avoid rescheduling dispite the evident lack of minecraft skill.

## Objective

Analyze team performance data across four modalities:

1. JSON behavior logs
2. Video recordings
3. Chat transcripts

Identify correlations between AI interventions and team outcomes.

## Note

All datasets were taken from official CHART ASIST Study 3 Dataset available at ASU official repository.

#### Subset used :

| Team ID | ASI ID        | trial   | intervention_recipent     |
| ------- | ------------- | ------- | ------------------------- |
| 000286  | ASI-CMURI-TA1 | T000829 | E001211, E001215, E001155 |



**AI Agent Action signals** -

1. RemindTransporterBeep
2. InformAboutTriagedVictim
3. RemindMedicToInformAboutTriagedVicti
4. TriageCriticalVictim
5. EvacuateCriticalVictim
6. EncouragePlayerProximityToMedicIHMCDyad
7. RemindChangeMarke
8. RemindRubblePerturbatio
9. EvacuationZoneDistanc
10. TeamSawVictimMarke
11. TimeElapse
12. StartEvacuatio

**Agents location**-

1. Location of the agent in the map -> Room Name

**Agent Action**-

1. Transporting victims
2. Performing their role task including stabilizing victims
3. wakening up critical victims
4. placing marking block-> Regular, A, B, C
5. placing marking block for threat rooms
6. removing rubbles
7. detecting victims

## Manual Dataframe

Timestamp, AI Message, AI Action Class, Transporter Message, Engineer Message, Medic Message is extracted from transcript.csv

We use Multimodal LLM analysis to analyze video data to give states and locations of agents + victims throughout the experiment


| Time Stamp (Transcript) | Asi Message (Transcript) | Asi Action Class (Transcript) | Transporter Message (Transcript) | Engineer Message (Transcript) | Medic Message (Transcript) |
| ----------------------- | ------------------------ | ----------------------------- | ------------------------------- | ----------------------------- | -------------------------- |
| 11:23:01                | N/A                      | N/A                           | N/A                             | N/A                           | N/A                        |

## llmv1 Dataframe

States tells us what the agent is doing
Locations tells use where the agent is located in the map

States and locations are fused together using LLM analysis to form one action_state column signifying their role in a situation

| timestamp (Transcript) | asi_reason (Transcript) | asi_action (Transcript) | transporter_message (Transcript) | engineer_message (Transcript) | medic_message (Transcript) | transporter_action_state (LLM) | engineer_action_state (LLM) | medic_action_state (LLM) | victim_location (LLM) | 
| --------- | --------- | --------- | ------------------ | --------------- | ------------ | --------------- | --------------- | ------------ | --------------- |
|           |           |           |                    |                 |              |                 |                 |              |                 |


We then utilize another LLM to finally provide ASI Advice score and team score for their actions and LLM's reasoning behiind that

## llmv2 Dataframe
| timestamp (Transcript) | asi_reason (Transcript) | asi_action (Transcript) | transporter_message (Transcript) | engineer_message (Transcript) | medic_message (Transcript) | transporter_action_state (LLM) | engineer_action_state (LLM) | medic_action_state (LLM) | victim_location (LLM) | team_score (LLM) | asi_advice_score (LLM) | team_score_reason (LLM) | asi_advice_score_reason (LLM)
| --------- | --------- | --------- | ------------------ | --------------- | ------------ | --------------- | ---------- | --------------- | --------------- |--------------- |--------------- | --------------- | --------------- |
|     22:03      |   You guys should do [asi_action_class] because...        | 1. RemindTransporterBeep <br/>2. InformAboutTriagedVictim <br/>3. RemindMedicToInformAboutTriagedVicti <br/>4. TriageCriticalVictim <br/>5. EvacuateCriticalVictim <br/>6.EncouragePlayerProximityToMedicIHMCDyad <br/>7. RemindChangeMarke <br/>8. RemindRubblePerturbatio <br/>9. EvacuationZoneDistanc <br/>10. TeamSawVictimMarke <br/>11. TimeElapse <br/>12. StartEvacuatio|     I'm coming for you medic               |     This is more important               |     I can't help you!               |     Carrying a victim from b4 to B2 room               |     Clearing rubbles in threat room for medic  at a9 room          |     waking up critical victim at g5 room        |    next to medic, far from engineer, close to transporter            |  40%          |     75%           | team was inconsistent with their tasks, especially... | asi's advice is particularly useful because... |


## AI Instruct Modal

We finetune pretrained LLM on our data for understanding minecraft test bed for asist thoroughly and deeply. Then we utilize it to perform-

1. **Multimodal DataAnalysis of Video Data** 

To give information about agent locations and their actions

2. **Text Fusion Data Analysis**

To fuse meanings and relationships between agent communication, their location and state in given situations to a single column

3. **Scoring Analysis**

To score humans and ASI's advice on team work communication and collaboration data   



# Installing dependencies

In [1]:
import cv2
from pathlib import Path
import json
import pandas as pd
# import openai
import matplotlib.pyplot as plt
import os
import warnings
# import torch
from dotenv import load_dotenv

warnings.filterwarnings("ignore")

# Load environment variables from .env file
load_dotenv()

# Fetch GEMINI_API_KEY
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Preparing Dataset

### 1. JSON Logs Processing
#### Objective
Extract structured data from nested JSON logs containing:
- Team actions
- AI intervention timestamps
- Mission outcomes


In [2]:


# def parse_json_logs(input_path: Path, output_path: Path) -> pd.DataFrame:
#     """Flatten nested JSON logs into structured format"""
#     with open(input_path, 'r') as f:
#         data = [json.loads(line) for line in f]
    
#     df = pd.json_normalize(data, sep='_')
#     df.to_csv(output_path, index=False)
#     return df

# # Process all trial messages
# input_files = [
#     Path("data/json_logs/HSRData_TrialMessages_Trial-T000603_..."),
#     Path("data/json_logs/HSRData_TrialMessages_Trial-T000639_..."),
#     Path("data/json_logs/HSRData_TrialMessages_Trial-T000671_...")
# ]

# output_dir = Path("data/processed/json_parsed/")
# output_dir.mkdir(parents=True, exist_ok=True)

# for file in input_files:
#     output_file = output_dir / f"{file.stem}_parsed.csv"
#     df = parse_json_logs(file, output_file)
#     print(f"Processed {len(df)} records from {file.name}")



# Preparing Model

### Objective 

Finetune the pretrained instruct model on testbed refined high quality data and information about agents, environment, locations. 


1. Transporting victims
2. Performing their role task including stabilizing victims
3. wakening up critical victims
4. placing marking block-> Regular, A, B, C
5. placing marking block for threat rooms
6. removing rubbles
7. detecting victims

![image.png](chart_study3_guide/blocks.png)


![image.png](chart_study3_guide/map.png)



[ Maybe : Train model to understand video data as well ]

### Engineer Instruct POV Model

Finetune the pretrained instruct model on testbed refined high quality data and information about agents actions in minecraft video. 

#### Possible Actions

1. Placing Avatar Yellow Marker Block for notifying there's a victim nearby for others to be aware. 
2. Placing Blank Yellow Marker Block for notifying there's a threat room nearby for others to be aware.
3. Destroying rubles for making way for rescue teammates

In [3]:
# Engineer POV model instruction (focusing only on in-game actions)
engineer_pov_model_instruction = """You are a professional Minecraft Agent Activity Logger. You only have to log what the agent is doing in their first-person view. DO NOT listen to what they're saying. Follow this guideline thoroughly to log any of these important details which are happening in the video every 3 seconds of the video. 

1. The Minecraft Agent POV Video:
 - The video shows what the Engineer agent is doing in first-person view. The Engineer's job is to work together with a medic and transporter to save victims in a Minecraft rescue simulation. Key events to document:
    a) Placing Avatar Yellow Marker Block for notifying there's a victim nearby
    b) Placing Blank Yellow Marker Block for notifying there's a threat room nearby
    c) Placing Avatar Red Marker Block for notifying there's a critical victim nearby
    d) Placing Avatar Blue Marker Block for notifying there's a regular victim nearby
    e) Placing Blank Red Marker Block for notifying there's rubble nearby
    f) Placing Blank Blue Marker Block for notifying there's a help request nearby
    g) Destroying rubbles for making way for rescue teammates
    h) Interacting with any blocks or items
    i) Moving through hallways or rooms
    j) Encountering other players (the Engineer wears blue skin, medic wears red, and the transporter wears green skin)

2. Logging Instructions:
 - Log the agent's activity every 3 seconds TILL THE VERY END OF THE VIDEO in JSON format
 - If no key event occurs, log that "nothing significant is happening"

"""

# Engineer Location model instruction (focusing only on map location)
engineer_location_model_instruction = """You are a professional Minecraft Agent Location tracker. You only have to track where the agent is on the map view. Follow this guideline thoroughly to log the location details every 3 seconds of the video.

1. The Location Map Video:
 - The video shows a map view with a blue dot representing the Engineer agent. Your job is to track this blue dot and report its position. Key points:
    a) Location of the agent throughout the map across various zones
    b) Movement between rooms or zones
    c) Time spent in specific locations

2. Map Details:
 - The map is divided into several zones labeled as N Zone A, N Zone B, N Zone C, S Zone A, S Zone B, and S Zone C
 - Each zone contains subzones labeled with alphanumeric identifiers (e.g., A1, B2, C3, etc.)
 - Key landmarks:
    - Yellow markers indicate victims or threats
    - Red markers indicate critical victims or rubble
    - Blue markers indicate regular victims or help requests

3. Logging Instructions:
 - Log the blue dot's position every 3 seconds TILL THE VERY END OF THE VIDEO in JSON format
 - Note any zone transitions or prolonged stays in one location

"""


print("\n", engineer_location_model_instruction, "\n", )



 You are a professional Minecraft Agent Location tracker. You only have to track where the agent is on the map view. Follow this guideline thoroughly to log the location details every 3 seconds of the video.

1. The Location Map Video:
 - The video shows a map view with a blue dot representing the Engineer agent. Your job is to track this blue dot and report its position. Key points:
    a) Location of the agent throughout the map across various zones
    b) Movement between rooms or zones
    c) Time spent in specific locations

2. Map Details:
 - The map is divided into several zones labeled as N Zone A, N Zone B, N Zone C, S Zone A, S Zone B, and S Zone C
 - Each zone contains subzones labeled with alphanumeric identifiers (e.g., A1, B2, C3, etc.)
 - Key landmarks:
    - Yellow markers indicate victims or threats
    - Red markers indicate critical victims or rubble
    - Blue markers indicate regular victims or help requests

3. Logging Instructions:
 - Log the blue dot's position

### Medic Instruct POV 

Finetune the pretrained instruct model on testbed refined high quality data and information about agents actions in minecraft video. 

1. Placing Avatar Yellow Marker Block for notifying there's a victim nearby for others to be aware. 
2. Placing Blank Yellow Marker Block for notifying there's a threat room nearby for others to be aware.
3. Stabilizing avatar victim blocks turning into safe blocks.
4. Stabilizing avatar victim blocks turning into X blocks notifying victim is dead.

In [4]:
medic_pov_model_instruction = """You are a professional Minecraft Medic Agent Activity analyzer. You only have to log what the agent is doing in their first-person view. DO NOT listen to what they're saying. Follow this guideline thoroughly to log any of these important details which are happening in the video every 3 seconds of the video.

1. The Minecraft Agent POV Video:
 - The video shows what the Medic agent is doing in first-person view. The Medic's job is to work together with an engineer and transporter to save victims in a Minecraft rescue simulation. Key events to document:
    a) Placing Avatar Yellow Marker Block for notifying there's a victim nearby
    b) Placing Blank Yellow Marker Block for notifying there's a threat room nearby
    c) Placing Avatar Red Marker Block for notifying there's a critical victim nearby
    d) Placing Avatar Blue Marker Block for notifying there's a regular victim nearby
    e) Stabilizing avatar victim blocks turning into safe blocks
    f) Stabilizing avatar victim blocks turning into X blocks notifying victim is dead
    g) Moving through hallways or rooms
    h) Encountering other players (the Medic wears red skin, engineer wears blue, and the transporter wears green skin)

2. Logging Instructions:
 - Log the agent's activity every 3 seconds TILL THE VERY END OF THE VIDEO
 - If no key event occurs, log that "nothing significant is happening"
"""

# New model instruction for location analysis
medic_location_model_instruction = """You are a professional Minecraft Agent Location tracker. You only have to track where the agent is on the map view. Follow this guideline thoroughly to log the location details every 3 seconds of the video.

1. The Location Map Video:
 - The video shows a map view with a red dot representing the Medic agent. Your job is to track this red dot and report its position. Key points:
    a) Location of the agent throughout the map across various zones
    b) Movement between rooms or zones
    c) Time spent in specific locations

2. Map Details:
 - The map is divided into several zones labeled as N Zone A, N Zone B, N Zone C, S Zone A, S Zone B, and S Zone C
 - Each zone contains subzones labeled with alphanumeric identifiers (e.g., A1, B2, C3, etc.)
 - Key landmarks:
    - Yellow markers indicate victims or threats
    - Red markers indicate critical victims or rubble
    - Blue markers indicate regular victims or help requests

3. Logging Instructions:
 - Log the red dot's position every 3 seconds in JSON format
 - Note any zone transitions or prolonged stays in one location
"""
print(medic_location_model_instruction)

You are a professional Minecraft Agent Location tracker. You only have to track where the agent is on the map view. Follow this guideline thoroughly to log the location details every 3 seconds of the video.

1. The Location Map Video:
 - The video shows a map view with a red dot representing the Medic agent. Your job is to track this red dot and report its position. Key points:
    a) Location of the agent throughout the map across various zones
    b) Movement between rooms or zones
    c) Time spent in specific locations

2. Map Details:
 - The map is divided into several zones labeled as N Zone A, N Zone B, N Zone C, S Zone A, S Zone B, and S Zone C
 - Each zone contains subzones labeled with alphanumeric identifiers (e.g., A1, B2, C3, etc.)
 - Key landmarks:
    - Yellow markers indicate victims or threats
    - Red markers indicate critical victims or rubble
    - Blue markers indicate regular victims or help requests

3. Logging Instructions:
 - Log the red dot's position every 3

### Transporter Instruct POV Model


Finetune the pretrained instruct model on testbed refined high quality data and information about agents actions in minecraft video. 


1. Placing Avatar Yellow Marker Block for notifying there's a victim nearby for others to be aware. 
2. Placing Blank Yellow Marker Block for notifying there's a threat room nearby for others to be aware.
3. Transporting victims from location to location.
4. Standing on wifi signal block to detect victims nearby.



In [9]:
transporter_pov_model_instruction = """You are a professional Minecraft Transporter Agent Activity analyzer. You only have to log what the agent is doing in their first-person view. DO NOT listen to what they're saying. Follow this guideline thoroughly to log any of these important details which are happening in the video every 3 seconds of the video.

1. The Minecraft Agent POV Video:
 - The video shows what the Transporter agent is doing in first-person view. The Transporter's job is to work together with an engineer and medic to save victims in a Minecraft rescue simulation. Key events to document:
    a) Placing Avatar Yellow Marker Block for notifying there's a victim nearby
    b) Placing Blank Yellow Marker Block for notifying there's a threat room nearby
    c) Placing Avatar Red Marker Block for notifying there's a critical victim nearby
    d) Placing Avatar Blue Marker Block for notifying there's a regular victim nearby
    e) Placing Blank Red Marker Block for notifying there's rubble nearby
    f) Placing Blank Blue Marker Block for notifying there's a help request nearby
    g) Transporting victims from location to location
    h) Standing on wifi signal block to detect victims nearby
    i) Moving through hallways or rooms
    j) Encountering other players (the Transporter wears green skin, engineer wears blue, and the medic wears red skin)

2. Logging Instructions:
 - Log the agent's activity every 3 seconds in JSON format
 - If no key event occurs, log that "nothing significant is happening"
"""

# New model instruction for location analysis
transporter_location_model_instruction = """You are a professional Minecraft Agent Location tracker. You only have to track where the agent is on the map view. Follow this guideline thoroughly to log the location details every 3 seconds of the video.

1. The Location Map Video:
 - The video shows a map view with a green dot representing the Transporter agent. Your job is to track this green dot and report its position. Key points:
    a) Location of the agent throughout the map across various zones
    b) Movement between rooms or zones
    c) Time spent in specific locations

2. Map Details:
 - The map is divided into several zones labeled as N Zone A, N Zone B, N Zone C, S Zone A, S Zone B, and S Zone C
 - Each zone contains subzones labeled with alphanumeric identifiers (e.g., A1, B2, C3, etc.)
 - Key landmarks:
    - Yellow markers indicate victims or threats
    - Red markers indicate critical victims or rubble
    - Blue markers indicate regular victims or help requests

3. Logging Instructions:
 - Log the green dot's position every 3 seconds in JSON format
 - Note any zone transitions or prolonged stays in one location

"""
print(transporter_pov_model_instruction)

You are a professional Minecraft Transporter Agent Activity analyzer. You only have to log what the agent is doing in their first-person view. DO NOT listen to what they're saying. Follow this guideline thoroughly to log any of these important details which are happening in the video every 3 seconds of the video.

1. The Minecraft Agent POV Video:
 - The video shows what the Transporter agent is doing in first-person view. The Transporter's job is to work together with an engineer and medic to save victims in a Minecraft rescue simulation. Key events to document:
    a) Placing Avatar Yellow Marker Block for notifying there's a victim nearby
    b) Placing Blank Yellow Marker Block for notifying there's a threat room nearby
    c) Placing Avatar Red Marker Block for notifying there's a critical victim nearby
    d) Placing Avatar Blue Marker Block for notifying there's a regular victim nearby
    e) Placing Blank Red Marker Block for notifying there's rubble nearby
    f) Placing Blank

### Player's Action State Fusor Model


Fuses player actions and their location in maps to form a rich action state data. 


In [6]:
action_state_fusor_model_instruction = """You are a professional context summarizer. Your job is to analyze a data having two properties which is action and state. You have to summarize the data into a single sentence. The data is about a minecraft test bed rescue simulation where three agents named Engineer, Medic and Transporter work and communicate with each other to rescue victims from threat rooms. Here is some information about the agents :\n1. Engineer : Engineer agent wears blue skin while medic wears red and transporter wears green skin. Key events- \na) Placing Avatar Yellow Marker Block for notifying there's a victim nearby for others to be aware. \nb)Placing Blank Yellow Marker Block for notifying there's a threat room nearby for others to be aware.\nc) Destroying rubles for making way for rescue teammates.\n2. Transporter :  Transporter agent wears green skin while engineer wears blue and medic wears red skin. Key events- \na) Placing Avatar Yellow Marker Block for notifying there's a victim nearby for others to be aware. \nb)Placing Blank Yellow Marker Block for notifying there's a threat room nearby for others to be aware.\nc) Transporting victims from location to location.\nd) Standing on wifi signal block to detect victims nearby.\n3. Medic : Medic agent wears red skin while engineer wears blue and transporter wears green skin. Key events- \na) Placing Avatar Yellow Marker Block for notifying there's a victim nearby for others to be aware. \nb)Placing Blank Yellow Marker Block for notifying there's a threat room nearby for others to be aware.\nc) Stabilizing avatar victim blocks turning into safe blocks\nd) Stabilizing avatar victim blocks turning into X blocks notifying victim is dead. \nHere are all important things to consider - \n- 1. Action : \n - The action is what the agent might be doing or did in their task during the environment mission. \n- 2. Communication : \n - The communication transmitted from agent in the video shared with either of medic, transporter or engineer. \n It's not neccessary that every second there will be a key event so you can just say that nothing is happening in the video. Example JSON Response -\n{timestamp: "00:03", "action_state":"Engineer went into A2 Zone to help medic and destroyed rubble"},{timestamp: "00:09", "action_state":"Medic was just communicating but did nothing from its side"}"""
action_state_fusor_model_instruction

'You are a professional context summarizer. Your job is to analyze a data having two properties which is action and state. You have to summarize the data into a single sentence. The data is about a minecraft test bed rescue simulation where three agents named Engineer, Medic and Transporter work and communicate with each other to rescue victims from threat rooms. Here is some information about the agents :\n1. Engineer : Engineer agent wears blue skin while medic wears red and transporter wears green skin. Key events- \na) Placing Avatar Yellow Marker Block for notifying there\'s a victim nearby for others to be aware. \nb)Placing Blank Yellow Marker Block for notifying there\'s a threat room nearby for others to be aware.\nc) Destroying rubles for making way for rescue teammates.\n2. Transporter :  Transporter agent wears green skin while engineer wears blue and medic wears red skin. Key events- \na) Placing Avatar Yellow Marker Block for notifying there\'s a victim nearby for others 

### Scoring Model

Consideres all important data and gives score to each intervention's team and asi advice with reasoning.

In [7]:
scoring_model_instruction = """You are a professional teamwork scorer. Your job is to analyze a data having various properties of the data to give . You have to rate the team work score to the team, You have to be impartial and a critique like a professional. The data is about a minecraft test bed rescue simulation where three agents named Engineer, Medic and Transporter work and communicate with each other to rescue victims from threat rooms. Here is some information about the agents :\n1. Engineer : Engineer agent wears blue skin while medic wears red and transporter wears green skin. Key events- \na) Placing Avatar Yellow Marker Block for notifying there's a victim nearby for others to be aware. \nb)Placing Blank Yellow Marker Block for notifying there's a threat room nearby for others to be aware.\nc) Destroying rubles for making way for rescue teammates.\n2. Transporter :  Transporter agent wears green skin while engineer wears blue and medic wears red skin. Key events- \na) Placing Avatar Yellow Marker Block for notifying there's a victim nearby for others to be aware. \nb)Placing Blank Yellow Marker Block for notifying there's a threat room nearby for others to be aware.\nc) Transporting victims from location to location.\nd) Standing on wifi signal block to detect victims nearby.\n3. Medic : Medic agent wears red skin while engineer wears blue and transporter wears green skin. Key events- \na) Placing Avatar Yellow Marker Block for notifying there's a victim nearby for others to be aware. \nb)Placing Blank Yellow Marker Block for notifying there's a threat room nearby for others to be aware.\nc) Stabilizing avatar victim blocks turning into safe blocks\nd) Stabilizing avatar victim blocks turning into X blocks notifying victim is dead. \nHere are all important things to consider - \n- 1. Action : \n - The action is what the agent might be doing or did in their task during the environment mission. \n- 2. Communication : \n - The communication transmitted from agent in the video shared with either of medic, transporter or engineer. Example JSON Response -\n{timestamp: "00:03", "team_score":"76", "reason:" "Engineer went into A2 Zone to help medic and destroyed rubble, Medic stabilized a victim and rushed to transporter, very quickly in this way the team work was good"}"""
scoring_model_instruction

'You are a professional teamwork scorer. Your job is to analyze a data having various properties of the data to give . You have to rate the team work score to the team, You have to be impartial and a critique like a professional. The data is about a minecraft test bed rescue simulation where three agents named Engineer, Medic and Transporter work and communicate with each other to rescue victims from threat rooms. Here is some information about the agents :\n1. Engineer : Engineer agent wears blue skin while medic wears red and transporter wears green skin. Key events- \na) Placing Avatar Yellow Marker Block for notifying there\'s a victim nearby for others to be aware. \nb)Placing Blank Yellow Marker Block for notifying there\'s a threat room nearby for others to be aware.\nc) Destroying rubles for making way for rescue teammates.\n2. Transporter :  Transporter agent wears green skin while engineer wears blue and medic wears red skin. Key events- \na) Placing Avatar Yellow Marker Bloc

# Data Analysis


## Transcript.csv analysis

In [10]:
df = pd.read_csv("/home/ash/CHART ASIST/data/transcripts/TM000297.csv")
df = df[1:]

In [11]:
df.head(5)

,Date,Timestamp,Intervention Message,Intervention Recipient,Speech Message,Medic,Transporter,Engineer,Explanation
1,2022-06-27,22:27:34,"Hello, I am ATLAS, and will be providing advic...","['E001001', 'E001146', 'E001064']",{},1,1,1,{'info': {'intervention_class': 'TeamWelcomeMe...
2,2022-06-27,22:27:55,{},{},people is be advised there is some wield damag...,0,0,1,{}
3,2022-06-27,22:28:06,{},{},okay so this is medic my plan is I'm going to ...,1,0,0,{}
4,2022-06-27,22:28:07,{},{},yes,0,1,0,{}
5,2022-06-27,22:28:10,{},{},copy,0,0,1,{}


#### Cleaning Data

Originally it had many columns and we reduced it down to only the ones which had data 



In [12]:
df["asi_message"] = df["intervention_message"].replace("{}", "")
df["team_message"] = df["speech_message"].replace("{}", "")

KeyError: 'intervention_message'

Adding a intervention_class column from explanation string

In [ ]:
# Extract intervention_class from explanation strings
df['intervention_class'] = df['explanation'].str.extract(
    r"'intervention_class'\s*:\s*'([^']*)'"
)

# Create binary columns for each unique intervention class
intervention_classes = df['intervention_class'].dropna().unique()
for cls in intervention_classes:
    df[cls] = df['intervention_class'].eq(cls).astype(int)

# Cleanup intermediate column
df = df.drop(columns=['intervention_class'])

# Clean column names by removing 'Intervention' suffix
df = df.rename(columns=lambda col: col[:-13] if col.endswith('Intervention') else col)



We don't need unique team ids, asi ids, date, trial id, intervention_recipent id or explanation since we have extracted the unique class

In [ ]:

df = df.drop(columns=["team","asi","date","intervention_recipent","intervention_message","speech_message","trial"], axis=1)



In [ ]:
df.sample(15)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

 Drop duplicated values

In [ ]:
df = df.drop_duplicates()

## Player's POV LLM Data Analysis - Multimodal

Using Gemini [Multimodal book](https://colab.research.google.com/drive/1eXv-lpq9Lp4s2Vf2ISmSEngMJnVMOBRk?usp=sharing#scrollTo=A0zpRFCI-weC) to setup instruct models for [analysis](https://aistudio.google.com/app/starter-apps/video).

In [ ]:
# Temp

df = pd.read_csv("/mnt/c/Users/Som/Desktop/CHART ASIST/temp.csv")


##### Player model test

In [10]:
# Test

import time
from pathlib import Path
from google import genai
from google.genai.types import GenerateContentConfig, HttpOptions

# Initialize the client
genai_client = genai.Client(api_key=GEMINI_API_KEY)
MODEL_ID = 'gemini-2.0-flash-exp'

# Define a test video path - use a specific video file that exists
test_video_path = Path("data/video/TM000297/experiment_1/engineer/pov/chunk_1/chunk_1.mp4")

# Test if the file exists
if not test_video_path.exists():
    print(f"Test video not found: {test_video_path}")
else:
    try:
        print(f"Testing with video: {test_video_path}")
        
        # Upload the video to Gemini
        file_upload = genai_client.files.upload(file=str(test_video_path))
        print(f"File uploaded: {file_upload.name}")
        
        # Wait for file to be processed (ACTIVE state)
        max_retries = 10
        retry_count = 0
        while retry_count < max_retries:
            # Check file status
            file_info = genai_client.files.get(name=file_upload.name)
            print(f"File state: {file_info.state}")
            
            if file_info.state == "ACTIVE":
                print(f"File is now active and ready for processing")
                break
            
            print("Waiting for file processing...")
            time.sleep(5)
            retry_count += 1
        
        if retry_count == max_retries:
            print("File processing timed out")
        else:
            # Generate content with a simplified instruction
            test_instruction = "Please Follow the guidelines. Here is the video. RESPOND IN JSON FORMAT ONLY."
            
            response = genai_client.models.generate_content(
                model=MODEL_ID,
                config=GenerateContentConfig(
                    system_instruction=[engineer_pov_model_instruction]
                ),
                contents=[
                    {"role": "user", "parts": [
                        {"file_data": {"file_uri": file_upload.uri, "mime_type": "video/mp4"}},
                        {"text": test_instruction}
                    ]}
                ]
            )
            response_text= response.text
            if "```json" in response_text:
                response_text = response.text.replace("```json", "").replace("```", "").strip()
            
            print(response_text)
    
    except Exception as e:
        print(f"Error in test script: {e}")

Testing with video: data/video/TM000297/experiment_1/engineer/pov/chunk_1/chunk_1.mp4
File uploaded: files/twxaf5271uyn
File state: FileState.PROCESSING
Waiting for file processing...
File state: FileState.ACTIVE
File is now active and ready for processing
Response received:
[
  {"timestamp": "00:00", "text": "Engineer is walking through a doorway"},
  {"timestamp": "00:03", "text": "Engineer is walking down the hallway"},
  {"timestamp": "00:06", "text": "Engineer is walking down the hallway"},
  {"timestamp": "00:09", "text": "Engineer is walking down the hallway"}
]


##### Location test

In [ ]:
# Test

import time
from pathlib import Path
from google import genai
from google.genai.types import GenerateContentConfig, HttpOptions

# Initialize the client
genai_client = genai.Client(api_key=GEMINI_API_KEY)
MODEL_ID = 'gemini-2.0-flash-exp'

# Define a test video path - use a specific video file that exists
test_video_path = Path("data/video/TM000297/experiment_1/engineer/location/chunk_1/chunk_1.mp4")

# Test if the file exists
if not test_video_path.exists():
    print(f"Test video not found: {test_video_path}")
else:
    try:
        print(f"Testing with video: {test_video_path}")
        
        # Upload the video to Gemini
        file_upload = genai_client.files.upload(file=str(test_video_path))
        print(f"File uploaded: {file_upload.name}")
        
        # Wait for file to be processed (ACTIVE state)
        max_retries = 10
        retry_count = 0
        while retry_count < max_retries:
            # Check file status
            file_info = genai_client.files.get(name=file_upload.name)
            print(f"File state: {file_info.state}")
            
            if file_info.state == "ACTIVE":
                print(f"File is now active and ready for processing")
                break
            
            print("Waiting for file processing...")
            time.sleep(5)
            retry_count += 1
        
        if retry_count == max_retries:
            print("File processing timed out")
        else:
            # Generate content with a simplified instruction
            test_instruction = "Please Follow the guidelines. Here is the video. RESPOND IN JSON FORMAT ONLY."
            
            response = genai_client.models.generate_content(
                model=MODEL_ID,
                config=GenerateContentConfig(
        system_instruction=[engineer_location_model_instruction
        ]
    ),
                contents=[
                    {"role": "user", "parts": [
                        {"file_data": {"file_uri": file_upload.uri, "mime_type": "video/mp4"}},
                        {"text": test_instruction}
                    ]}
                ]
            )
            response_text = response.text
            if "```json" in response_text:
                response_text = response.text.replace("```json", "").replace("```", "").strip()
            
            print("Response received:")
            print(response_text)
    
    except Exception as e:
        print(f"Error in test script: {e}")


### Engineer POV Instruct

#### POV Model

In [14]:
from typing import List
import time
from google import genai
import json
from pathlib import Path
from google.genai.types import GenerateContentConfig, HttpOptions

# Initialize the client with extended timeout
genai_client = genai.Client(
    api_key=GEMINI_API_KEY,
    http_options=HttpOptions(timeout=600)  # Extend timeout to 5 minutes
)
MODEL_ID = 'gemini-2.0-flash-exp'

# Define the path to the engineer folder
engineer_folder = Path("data/video/TM000297/experiment_1/engineer/pov")

# Loop over the chunks in the engineer folder
for chunk_folder in engineer_folder.iterdir():
    if chunk_folder.is_dir():
        # Check if response already exists
        response_file = chunk_folder / "pov.json"
        if response_file.exists():
            print(f"Response already exists for {chunk_folder.name}, skipping...")
            continue
            
        # Get the video file from the chunk folder
        video_file = next(chunk_folder.glob("*.mp4"), None)
        if video_file:
            print(f"Processing {video_file}")
            
            try:
                # Upload the video to Gemini
                try:
                    file_upload = genai_client.files.upload(file=video_file)
                    print(f"File uploaded: {file_upload.name}")
                except Exception as upload_error:
                    print(f"Error uploading file: {upload_error}")
                    break
                # Wait for file to be processed (ACTIVE state)
                max_retries = 10  # Increase max retries
                retry_count = 0
                while retry_count < max_retries:
                    # Check file status
                    file_info = genai_client.files.get(name=file_upload.name)
                    print(f"File state: {file_info.state}")
                    
                    if file_info.state == "ACTIVE":
                        print(f"File is now active and ready for processing")
                        break
                    
                    print("Waiting for file processing...")
                    time.sleep(5)
                    retry_count += 1
                
                if retry_count == max_retries:
                    print("File processing timed out")
                else:
                    # Generate content with a simplified instruction
                    test_instruction = "Please Follow the guidelines. Here is the video. RESPOND IN JSON FORMAT ONLY."
                    
                    response = genai_client.models.generate_content(
                        model=MODEL_ID,
                        config=GenerateContentConfig(
                            system_instruction=[engineer_pov_model_instruction]
                        ),
                        contents=[
                            {"role": "user", "parts": [
                                {"file_data": {"file_uri": file_upload.uri, "mime_type": "video/mp4"}},
                                {"text": test_instruction}
                            ]}
                        ]
                    )
                    response_text= response.text
                    if "```json" in response_text:
                        response_text = response.text.replace("```json", "").replace("```", "").strip()
                    
                    with open(response_file, "w") as f:
                        f.write(response_text)
                    print(f"Saved response to {response_file}")
                    # Add a cool-down period between API calls
                    print("Cooling down for 30 seconds before next request...")
                    
            except Exception as generate_error:
                print(f"Error generating content: {generate_error}")
                # Try to delete the file to free up resources
                try:
                    genai_client.files.delete(name=file_upload.name)
                    print(f"Deleted file {file_upload.name} after error")
                except:
                    pass
                time.sleep(60)  # Wait longer before retrying another file
            
            except Exception as e:
                print(f"Error processing {video_file}: {e}")
                break


Processing data/video/TM000297/experiment_1/engineer/pov/chunk_1/chunk_1.mp4
File uploaded: files/n3cwe1hg5je5
File state: FileState.PROCESSING
Waiting for file processing...
File state: FileState.ACTIVE
File is now active and ready for processing
Error generating content: The read operation timed out


KeyboardInterrupt: 

#### Location Model

In [ ]:
from typing import List
import time
from google import genai
import json

# Initialize the client
genai_client = genai.Client(api_key=GEMINI_API_KEY)
MODEL_ID = 'gemini-2.0-flash-exp'

# Define the path to the engineer folder
engineer_folder = Path("data/video/TM000297/experiment_1/engineer/location")

# Loop over the chunks in the engineer folder
for chunk_folder in engineer_folder.iterdir():
    if chunk_folder.is_dir():
        # Get the video file from the chunk folder
        video_file = next(chunk_folder.glob("*.mp4"), None)
        if video_file:
            print(f"Processing {video_file}")
            
            try:
                # Upload the video to Gemini
                with open(video_file, "rb") as f:
                    file_bytes = f.read()
                
                file_upload = genai_client.files.upload(file=str(video_file))
                
                # Wait for file to be processed (ACTIVE state)
                max_retries = 10
                retry_count = 0
                while retry_count < max_retries:
                    # Check file status
                    file_info = genai_client.files.get(name=file_upload.name)
                    if file_info.state == "ACTIVE":
                        print(f"File {file_upload.name} is now active and ready")
                        break
                    
                    print(f"File {file_upload.name} is in {file_info.state} state. Waiting...")
                    time.sleep(5)  # Wait 5 seconds before checking again
                    retry_count += 1
                
                if retry_count == max_retries:
                    print(f"File {file_upload.name} did not become active. Skipping.")
                    continue
                
                # Generate content using the uploaded file and the instruction
                response = genai_client.models.generate_content(
                    model=MODEL_ID,
                    config=GenerateContentConfig(
                        system_instruction=[engineer_location_model_instruction]
                    ),
                    contents=[
                        {"role": "user", "parts": [
                            {"file_data": {"file_uri": file_upload.uri, "mime_type": "video/mp4"}},
                            {"text": "Please Follow the guidelines. Here is the video. RESPOND IN JSON FORMAT ONLY."}
                        ]}
                    ]
                )
                
                response_text = response.text
                
                # Remove markdown code block syntax if present
                if "```json" in response_text:
                    response_text = response_text.replace("```json", "").replace("```", "").strip()
                
                # Save the response to the same chunk folder as a JSON file
                response_file = chunk_folder / "location.json"
                with open(response_file, "w") as f:
                    json.dump(response_text, f)
                print(f"Saved response to {response_file}")
            
            except Exception as e:
                print(f"Error processing {video_file}: {e}")


### Medic POV Instruct

#### POV Model

In [ ]:
from typing import List
import time
from google import genai
import json

# Initialize the client
genai_client = genai.Client(api_key=GEMINI_API_KEY)
MODEL_ID = 'gemini-2.0-flash-exp'

# Define the path to the engineer folder
medic_folder = Path("data/video/TM000297/experiment_1/medic/pov")

# Loop over the chunks in the engineer folder
for chunk_folder in medic_folder.iterdir():
    if chunk_folder.is_dir():
        # Get the video file from the chunk folder
        video_file = next(chunk_folder.glob("*.mp4"), None)
        if video_file:
            print(f"Processing {video_file}")
            
            try:
                # Upload the video to Gemini
                with open(video_file, "rb") as f:
                    file_bytes = f.read()
                
                file_upload = genai_client.files.upload(file=str(video_file))
                
                # Wait for file to be processed (ACTIVE state)
                max_retries = 10
                retry_count = 0
                while retry_count < max_retries:
                    # Check file status
                    file_info = genai_client.files.get(name=file_upload.name)
                    if file_info.state == "ACTIVE":
                        print(f"File {file_upload.name} is now active and ready")
                        break
                    
                    print(f"File {file_upload.name} is in {file_info.state} state. Waiting...")
                    time.sleep(5)  # Wait 5 seconds before checking again
                    retry_count += 1
                
                if retry_count == max_retries:
                    print(f"File {file_upload.name} did not become active. Skipping.")
                    continue
                
                # Generate content using the uploaded file and the instruction
                response = genai_client.models.generate_content(
                    model=MODEL_ID,
                    config=GenerateContentConfig(
                        system_instruction=[medic_pov_model_instruction]
                    ),
                    contents=[
                        {"role": "user", "parts": [
                            {"file_data": {"file_uri": file_upload.uri, "mime_type": "video/mp4"}},
                            {"text": "Please Follow the guidelines. Here is the video. RESPOND IN JSON FORMAT ONLY."}
                        ]}
                    ]
                )
                
                response_text = response.text
                
                # Remove markdown code block syntax if present
                if "```json" in response_text:
                    response_text = response_text.replace("```json", "").replace("```", "").strip()
                
                # Save the response to the same chunk folder as a JSON file
                response_file = chunk_folder / "pov.json"
                with open(response_file, "w") as f:
                    json.dump(response_text, f)
                print(f"Saved response to {response_file}")
            
            except Exception as e:
                print(f"Error processing {video_file}: {e}")


#### Location Model

In [ ]:
from typing import List
import time
from google import genai
import json

# Initialize the client
genai_client = genai.Client(api_key=GEMINI_API_KEY)
MODEL_ID = 'gemini-2.0-flash-exp'

# Define the path to the engineer folder
medic_folder = Path("data/video/TM000297/experiment_1/medic/location")

# Loop over the chunks in the engineer folder
for chunk_folder in medic_folder.iterdir():
    if chunk_folder.is_dir():
        # Get the video file from the chunk folder
        video_file = next(chunk_folder.glob("*.mp4"), None)
        if video_file:
            print(f"Processing {video_file}")
            
            try:
                # Upload the video to Gemini
                with open(video_file, "rb") as f:
                    file_bytes = f.read()
                
                file_upload = genai_client.files.upload(file=str(video_file))
                
                # Wait for file to be processed (ACTIVE state)
                max_retries = 10
                retry_count = 0
                while retry_count < max_retries:
                    # Check file status
                    file_info = genai_client.files.get(name=file_upload.name)
                    if file_info.state == "ACTIVE":
                        print(f"File {file_upload.name} is now active and ready")
                        break
                    
                    print(f"File {file_upload.name} is in {file_info.state} state. Waiting...")
                    time.sleep(5)  # Wait 5 seconds before checking again
                    retry_count += 1
                
                if retry_count == max_retries:
                    print(f"File {file_upload.name} did not become active. Skipping.")
                    continue
                
                # Generate content using the uploaded file and the instruction
                response = genai_client.models.generate_content(
                    model=MODEL_ID,
                    config=GenerateContentConfig(
                        system_instruction=[medic_location_model_instruction]
                    ),
                    contents=[
                        {"role": "user", "parts": [
                            {"file_data": {"file_uri": file_upload.uri, "mime_type": "video/mp4"}},
                            {"text": "Please Follow the guidelines. Here is the video. RESPOND IN JSON FORMAT ONLY."}
                        ]}
                    ]
                )
                
                response_text = response.text
                
                # Remove markdown code block syntax if present
                if "```json" in response_text:
                    response_text = response_text.replace("```json", "").replace("```", "").strip()
                
                # Save the response to the same chunk folder as a JSON file
                response_file = chunk_folder / "location.json"
                with open(response_file, "w") as f:
                    json.dump(response_text, f)
                print(f"Saved response to {response_file}")
            
            except Exception as e:
                print(f"Error processing {video_file}: {e}")


### Transporter POV Instruct

#### POV Model

In [ ]:
from typing import List
import time
from google import genai
import json

# Initialize the client
genai_client = genai.Client(api_key=GEMINI_API_KEY)
MODEL_ID = 'gemini-2.0-flash-exp'

# Define the path to the engineer folder
transporter_folder = Path("data/video/TM000297/experiment_1/transporter/pov")

# Loop over the chunks in the engineer folder
for chunk_folder in transporter_folder.iterdir():
    if chunk_folder.is_dir():
        # Get the video file from the chunk folder
        video_file = next(chunk_folder.glob("*.mp4"), None)
        if video_file:
            print(f"Processing {video_file}")
            
            try:
                # Upload the video to Gemini
                with open(video_file, "rb") as f:
                    file_bytes = f.read()
                
                file_upload = genai_client.files.upload(file=str(video_file))
                
                # Wait for file to be processed (ACTIVE state)
                max_retries = 10
                retry_count = 0
                while retry_count < max_retries:
                    # Check file status
                    file_info = genai_client.files.get(name=file_upload.name)
                    if file_info.state == "ACTIVE":
                        print(f"File {file_upload.name} is now active and ready")
                        break
                    
                    print(f"File {file_upload.name} is in {file_info.state} state. Waiting...")
                    time.sleep(5)  # Wait 5 seconds before checking again
                    retry_count += 1
                
                if retry_count == max_retries:
                    print(f"File {file_upload.name} did not become active. Skipping.")
                    continue
                
                # Generate content using the uploaded file and the instruction
                response = genai_client.models.generate_content(
                    model=MODEL_ID,
                    config=GenerateContentConfig(
                        system_instruction=[transporter_pov_model_instruction]
                    ),
                    contents=[
                        {"role": "user", "parts": [
                            {"file_data": {"file_uri": file_upload.uri, "mime_type": "video/mp4"}},
                            {"text": "Please Follow the guidelines. Here is the video. RESPOND IN JSON FORMAT ONLY."}
                        ]}
                    ]
                )
                response_text = response.text
                
                # Remove markdown code block syntax if present
                if "```json" in response_text:
                    response_text = response_text.replace("```json", "").replace("```", "").strip()
                # Save the response to the same chunk folder as a JSON file
                response_file = chunk_folder / "pov.json"
                with open(response_file, "w") as f:
                    json.dump(response_text, f)
                print(f"Saved response to {response_file}")
            
            except Exception as e:
                print(f"Error processing {video_file}: {e}")


#### Location Model

In [ ]:
from typing import List
import time
from google import genai
import json

# Initialize the client
genai_client = genai.Client(api_key=GEMINI_API_KEY)
MODEL_ID = 'gemini-2.0-flash-exp'

# Define the path to the engineer folder
transporter_folder = Path("data/video/TM000297/experiment_1/transporter/location")

# Loop over the chunks in the engineer folder
for chunk_folder in transporter_folder.iterdir():
    if chunk_folder.is_dir():
        # Get the video file from the chunk folder
        video_file = next(chunk_folder.glob("*.mp4"), None)
        if video_file:
            print(f"Processing {video_file}")
            
            try:
                # Upload the video to Gemini
                with open(video_file, "rb") as f:
                    file_bytes = f.read()
                
                file_upload = genai_client.files.upload(file=str(video_file))
                
                # Wait for file to be processed (ACTIVE state)
                max_retries = 10
                retry_count = 0
                while retry_count < max_retries:
                    # Check file status
                    file_info = genai_client.files.get(name=file_upload.name)
                    if file_info.state == "ACTIVE":
                        print(f"File {file_upload.name} is now active and ready")
                        break
                    
                    print(f"File {file_upload.name} is in {file_info.state} state. Waiting...")
                    time.sleep(5)  # Wait 5 seconds before checking again
                    retry_count += 1
                
                if retry_count == max_retries:
                    print(f"File {file_upload.name} did not become active. Skipping.")
                    continue
                
                # Generate content using the uploaded file and the instruction
                response = genai_client.models.generate_content(
                    model=MODEL_ID,
                    config=GenerateContentConfig(
                        system_instruction=[transporter_location_model_instruction]
                    ),
                    contents=[
                        {"role": "user", "parts": [
                            {"file_data": {"file_uri": file_upload.uri, "mime_type": "video/mp4"}},
                            {"text": "Please Follow the guidelines. Here is the video. RESPOND IN JSON FORMAT ONLY."}
                        ]}
                    ]
                )
                
                response_text = response.text
                
                # Remove markdown code block syntax if present
                if "```json" in response_text:
                    response_text = response_text.replace("```json", "").replace("```", "").strip()
                
                # Save the response to the same chunk folder as a JSON file
                response_file = chunk_folder / "location.json"
                with open(response_file, "w") as f:
                    json.dump(response_text, f)
                print(f"Saved response to {response_file}")
            
            except Exception as e:
                print(f"Error processing {video_file}: {e}")


## Extract JSON action_states to dataframe

In [ ]:
df_action_states = pd.DataFrame(["timestamp","engineer_action_state","medic_action_state","transporter_action_state"])
# merge df_action_states to df dataframe

## Fusion LLM Text Analysis



### Objective

Use LLM to analyze each agent's communication, location and state to form one [agent]_action_state 

**1. Map State**-

Location of the agent in the map -> Room Name

**2. Agent Action**-

1. chat
2. door
3. ItemDrop
4. ItemEquipped
5. ItemPickup
6. ItemUsed
7. Lever
8. PlayerJumped
9. PlayerSprinting
10. Scoreboard
11. Triage
12. RoleSelected
13. ProximityBlockInteraction
14. PlayerFrozenStateChange
15. ToolUsed
16. CompetencyTask
17. TrainingTask
18. ToolDepleted
19. MarkerPlaced
20. MarkerRemoved
21. RubbleCollapse
22. VictimEvacuated
23. VictimPickedUp
24. VictimPlaced
25. RubbleDestroyed
26. Signal
27. Pause
28. MissionState
29. Perturbation
30. PlanningStage
31. PerturbationRubbleLocations
32. location
33. proximity
34. dyad
35. VictimsExpired
36. PuzzleTextSummary
37. dialogue_event
38. TrialState

**3. Agent Message**-

Raw message extracted from transcript.csv

### Prompt for AI Modal 

You are 

### Response Type

```json
{
    agent: "engineer | transporter | medic | victim", [agent]_action_state " [Agent] was likely doing this at this location " 
}
```

### Pass each row to LLM and retreive JSON

In [ ]:
# TODO 

### Convert JSON to a single dataframe of csv -> [agent]_action_state and append it to csv

In [ ]:
# TODO

## Score LLM Text Analysis



### Objective

Use LLM to analyze agent's advice and team's communication and collaboration to form team_score and asi_advice score

| timestamp | asi_reason | asi_action | transporter_message | engineer_message | medic_message | transporter_action_state | engineer_action_state | medic_action_state | victim_location | 
| --------- | --------- | --------- | ------------------ | --------------- | ------------ | --------------- | --------------- | ------------ | --------------- |
|           |           |           |                    |                 |              |                 |                 |              |                 |

### Prompt for AI Modal 

You are 

### Response Type

```json
{
    asi_advice_score: "number %", team_score : "number %" 
}
```

### Pass each row to LLM and retreive JSON

In [ ]:
# TODO 

### Convert JSON to a single dataframe of csv -> team_score and asi_advice score and append it to csv

In [ ]:
# TODO